## Problem 3: How far did people travel? (8 points)

During this task, the aim is to calculate the (air-line) distance in meters that each social media user in the data set prepared in *Problem 2* has travelled in-between the posts. We’re interested in the Euclidean distance between subsequent points generated by the same user.

For this, we will need to use the `userid` column of the data set `kruger_posts.shp` that we created in *Problem 2*.

Answer the following questions:
- What was the shortest distance a user travelled between all their posts (in meters)?
- What was the mean distance travelled per user (in meters)?
- What was the maximum distance a user travelled (in meters)?

---


### a) Read the input file and re-project it

- Read the input file `kruger_points.shp` into a geo-data frame `kruger_points`
- Transform the data from WGS84 to an `EPSG:32735` projection (UTM Zone 35S, suitable for South Africa). This CRS has *metres* as units.

In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely import LineString
import pathlib

/tmp/ipykernel_1900/1598201047.py:1: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [2]:
# ADD YOUR OWN CODE HERE
kruger_points = gpd.read_file("data/kruger_points.shp")

In [3]:
kruger_points = kruger_points.to_crs(epsg=32735)

In [4]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

# Check the data
kruger_points.head()

,lat,lon,timestamp,userid,geometry
0,-24.980792,31.484633,2015-07-07 03:02,66487960,POINT (-4695752.719 14973674.275)
1,-25.499225,31.508906,2015-07-07 03:18,65281761,POINT (-4748939.258 15014098.837)
2,-24.342578,30.930866,2015-03-07 03:38,90916112,POINT (-4672729.591 14859391.193)
3,-24.854614,31.519718,2015-10-07 05:04,37959089,POINT (-4679391.656 14969037.444)
4,-24.921069,31.520836,2015-10-07 05:19,27793716,POINT (-4686373.982 14973910.589)


In [5]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

# Check that the crs is correct after re-projecting (should be epsg:32735)
import pyproj
assert kruger_points.crs == pyproj.CRS("EPSG:32735")

### b) Group the data by user id

Group the data by `userid` and store the grouped data in a variable `grouped_by_users`

In [6]:
# ADD YOUR OWN CODE HERE

In [7]:
grouped_by_users = kruger_points.groupby("userid")
print(len(grouped_by_users.userid))
grouped_by_users.userid.count().sort_values(ascending=False).head()

14990


userid
54796261    457
49434843    323
86267392    313
7619876     296
21981793    293
Name: userid, dtype: int64

In [8]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

# Check the number of groups:
assert len(grouped_by_users.groups) == kruger_points["userid"].nunique(), "Number of groups should match number of unique users!"

### c) Create `shapely.geometry.LineString` objects for each user connecting the points from oldest to most recent

There are multiple ways to solve this problem (see the [hints for this exercise](https://autogis-site.readthedocs.io/en/latest/lessons/lesson-2/exercise-2.html). You can use, for instance, a dictionary or an empty GeoDataFrame to collect data that is generated using the steps below:

- Use a for-loop to iterate over the grouped object. For each user’s data: 
    - [sort](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sort_values.html) the rows by timestamp 
    - create a `shapely.geometry.LineString` based on the user’s points

**CAREFUL**: Remember that every LineString needs at least two points. Skip users who have less than two posts.

Store the results in a `geopandas.GeoDataFrame` called `movements`, and remember to assign a CRS.

In [9]:
# ADD YOUR OWN CODE HERE

# Filtered userid's with less than 2 points
filtered_kruger_points = grouped_by_users.filter(lambda x: len(x) >= 2)

data = {
    "userid": [],
    "user_linestring": []
}

for key, item in filtered_kruger_points.groupby("userid"):
    data["userid"].append(key)
    sorted = item.sort_values(by="timestamp").drop(item.index[1:-1])
    data["user_linestring"].append(LineString(sorted.geometry.values))

movements = gpd.GeoDataFrame(data, geometry='user_linestring', crs="4326")

In [10]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

# Check the result
print(type(movements))
print(movements.crs)

movements

<class 'geopandas.geodataframe.GeoDataFrame'>
EPSG:4326


,userid,user_linestring
0,16301,"LINESTRING (-4684246.01511 14939886.37792, -46..."
1,45136,"LINESTRING (-4770692.22977 14940874.44922, -47..."
2,50136,"LINESTRING (-4687987.86590 14987928.78204, -46..."
3,88775,"LINESTRING (-4773713.34517 14938272.13166, -47..."
4,88918,"LINESTRING (-4699374.15877 14988142.85839, -46..."
...,...,...
9021,99921781,"LINESTRING (-4769055.74187 14937157.52724, -47..."
9022,99936874,"LINESTRING (-4688007.08900 14987931.59487, -46..."
9023,99964140,"LINESTRING (-4636958.08148 14905786.10960, -46..."
9024,99986933,"LINESTRING (-4638612.17200 14901687.48834, -46..."


### d) Calculate the distance between all posts of a user

- Check once more that the CRS of the data frame is correct
- Compute the lengths of the lines, and store it in a new column called `distance`

In [11]:
# ADD YOUR OWN CODE HERE
movements["distance"] = movements["user_linestring"].apply(lambda x: x.length)

In [12]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

#Check the output
movements.head()

,userid,user_linestring,distance
0,16301,"LINESTRING (-4684246.01511 14939886.37792, -46...",97338.713788
1,45136,"LINESTRING (-4770692.22977 14940874.44922, -47...",0.000000
2,50136,"LINESTRING (-4687987.86590 14987928.78204, -46...",0.000000
3,88775,"LINESTRING (-4773713.34517 14938272.13166, -47...",0.095916
4,88918,"LINESTRING (-4699374.15877 14988142.85839, -46...",11388.305140


### e) Answer the original questions

You should now be able to quickly find answers to the following questions: 
- What was the shortest distance a user travelled between all their posts (in meters)? (store the value in a variable `shortest_distance`)
- What was the mean distance travelled per user (in meters)? (store the value in a variable `mean_distance`)
- What was the maximum distance a user travelled (in meters)? (store the value in a variable `longest_distance`)

In [13]:
# ADD YOUR OWN CODE HERE
shortest_distance = movements["distance"].min().round(0) # 0 meters
mean_distance = movements["distance"].mean().round(0)  # 25929 meters
longest_distance = movements["distance"].max().round(0) # 365398 meters

### f) Save the movements in a file

Save the `movements` into a new Shapefile called `movements.shp` inside the `data` directory.

In [14]:
# ADD YOUR OWN CODE HERE
DATA_DIRECTORY = "data"
DATA_DIRECTORY = pathlib.Path("data")

movements.to_file(DATA_DIRECTORY / "movements.shp")

/tmp/ipykernel_1900/4183956191.py:5: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  movements.to_file(DATA_DIRECTORY / "movements.shp")


In [15]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

assert (DATA_DIRECTORY / "movements.shp").exists()


---

# Fantastic job!

That’s all for this week! 